In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
! pip install tensorflow_addons
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're u

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.12.0
Running on TPU  ['10.116.119.162:8470']


## Load the Data for training

In [ ]:
def parse(serialized,signal_shape=(12,1024),label_shape=(1,)):

    features = {'signal': tf.io.FixedLenFeature([], tf.string), 'label': tf.io.FixedLenFeature([], tf.int64)}
    parsed_example = tf.io.parse_single_example(serialized=serialized, features=features)

    signal = parsed_example['signal']
    label = parsed_example['label']

    signal = tf.io.decode_raw(signal, tf.float64)
    signal = tf.reshape(signal, shape=signal_shape)
    signal = tf.transpose(signal)
    signal = signal*(10**5)

    # label = tf.io.decode_raw(label, tf.int64)
    label = tf.reshape(label, shape=label_shape)

    return signal, label

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
train_files = tf.io.matching_files('gs://tinyml/fyp_train_v1/*train.tfrec')
train_files = tf.random.shuffle(train_files)
shards = tf.data.Dataset.from_tensor_slices(train_files)
train_dataset = shards.interleave(lambda x: tf.data.TFRecordDataset(x), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.map(parse, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(128)
train_dataset = train_dataset.prefetch(AUTOTUNE)

AUTOTUNE=tf.data.AUTOTUNE
test_files = tf.io.matching_files('gs://tinyml/fyp_test_v1/*test.tfrec')
shards = tf.data.Dataset.from_tensor_slices(test_files)
test_dataset = shards.interleave(lambda x: tf.data.TFRecordDataset(x), num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(parse, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(128)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [ ]:
# for i in train_dataset:
#   print(i[0][0].shape)
#   print(i[1][0][0])
#   plt.plot(i[0][0][:,0])
#   break

pos = 0
tot = 0
for i in train_dataset:
  # print(i[1].shape)
  pos+=sum(i[1])
  tot+=i[1].shape[0]
  # break

pos = pos.numpy()[0]
print(pos, tot)

Exception ignored in: <function Executor.__del__ at 0x797b781b89d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.OutOfRangeError: End of sequence


6928 10000


## Large model simillar to RestNet50

In [ ]:
def block_type1(n_filters1,n_filters2, kernel, pad, x):
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(x)
    block1=layers.BatchNormalization()(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.BatchNormalization()(block1)
    block1=layers.SpatialDropout1D(0.2)(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(block1)
    block1=layers.BatchNormalization()(block1)
    block1=layers.SpatialDropout1D(0.2)(block1)

    block1_o=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(x)
    block1_o=layers.BatchNormalization()(block1_o)
    block1_o=layers.SpatialDropout1D(0.2)(block1_o)
    added1=layers.add([block1,block1_o])
    return layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(added1)

def block_type2(n_filters1,n_filters2, kernel, pad, x):
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(x)
    block=layers.BatchNormalization()(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.BatchNormalization()(block)
    block=layers.SpatialDropout1D(0.2)(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(block)
    block=layers.BatchNormalization()(block)
    block=layers.SpatialDropout1D(0.2)(block)

    added=layers.add([x,block])
    return layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(added)

In [ ]:
def create_model():
  Input=keras.Input(shape=(1024,12),name="Input_signal")
  x=layers.Conv1D(32,3,padding="same",activation='swish')(Input)
  x=layers.BatchNormalization()(x)
  x=layers.Conv1D(32,3,padding="same",activation='swish')(x)
  x=layers.MaxPool1D(2)(x)

  x1=block_type1(32,128,3,"same",x)
  x2=block_type2(32,128,3,"same",x1)
  x3=block_type2(32,128,3,"same",x2)
  x3=layers.AveragePooling1D(4)(x3)
  x4=block_type1(64,256,7,"same",x3)
  x5=block_type2(64,256,7,"same",x4)
  x6=block_type2(64,256,7,"same",x5)
  x7=block_type2(64,256,7,"same",x6)
  x7=layers.AveragePooling1D(4)(x7)
  x8=block_type1(128,512,9,"same",x7)
  x9=block_type2(128,512,9,"same",x8)
  x10=block_type2(128,512,9,"same",x9)
  x11=block_type2(128,512,9,"same",x10)
  x12=block_type2(128,512,9,"same",x11)
  x13=block_type2(128,512,9,"same",x12)
  x13=layers.AveragePooling1D(4)(x13)
  x14=block_type1(256,1024,11,"same",x13)
  x15=block_type2(256,1024,11,"same",x14)
  x16=block_type2(256,1024,11,"same",x15)
  x16=layers.GlobalAveragePooling1D()(x16)
  x16=layers.Dense(16,activation='swish')(x16)
  x16=layers.Dense(1)(x16)

  model=keras.Model(Input,x16)
  F1 = tfa.metrics.FBetaScore(num_classes=1)
  AUROC = tf.keras.metrics.AUC(curve='ROC', name = 'AUROC')
  AUPRC = tf.keras.metrics.AUC(curve='PR', name = 'AUPRC')
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy', F1, AUROC, AUPRC])
  keras.utils.plot_model(model,'model.png',show_layer_activations=True,show_shapes=True)
  model.summary()
  return model

with tpu_strategy.scope():
  model = create_model()


In [ ]:
keras.utils.plot_model(model,'/content/drive/MyDrive/FYP_ML/model1.png',show_shapes=True)

In [ ]:
model.fit(train_dataset, validation_data = test_dataset, epochs=100)

## Working large model

In [ ]:
def block_type1(n_filters1,n_filters2, kernel, pad, x):
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(x)
    block1=layers.BatchNormalization()(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.BatchNormalization()(block1)
    block1=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block1)
    block1=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(block1)
    block1=layers.BatchNormalization()(block1)

    block1_o=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(x)
    block1_o=layers.BatchNormalization()(block1_o)
    added1=layers.add([block1,block1_o])
    return layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(added1)

def block_type2(n_filters1,n_filters2, kernel, pad, x):
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(x)
    block=layers.BatchNormalization()(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.BatchNormalization()(block)
    block=layers.Conv1D(n_filters1,kernel,padding=pad,activation='swish')(block)
    block=layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(block)
    block=layers.BatchNormalization()(block)

    added=layers.add([x,block])
    return layers.Conv1D(n_filters2,kernel,padding=pad,activation='swish')(added)

In [ ]:
def create_model():
  Input=keras.Input(shape=(1024,12),name="Input_signal")
  x=layers.Conv1D(32,3,padding="same",activation='swish')(Input)
  y=layers.Conv1D(32,5,padding='same',activation='swish')(Input)
  a=layers.add([x,y])
  # x=layers.AveragePooling1D(2)(a)
  x=layers.MaxPooling1D(4)(a)

  x1=block_type1(32,128,3,"same",x)
  x2=block_type2(32,128,13,"same",x1)
  x3=block_type2(32,128,23,"same",x2)
  x3=layers.AveragePooling1D(4)(x3)
  x4=block_type1(64,256,7,"same",x3)
  x5=block_type2(64,256,17,"same",x4)
  x6=block_type2(64,256,27,"same",x5)
  x7=block_type2(64,256,7,"same",x6)
  x16=layers.GlobalAveragePooling1D()(x7)
  x16=layers.Dense(256)(x16)
  x16=layers.Dropout(0.5)(x16)
  x16=layers.Dense(16,activation='swish')(x16)
  x16=layers.Dense(1,activation='sigmoid')(x16)

  model=keras.Model(Input,x16)
  F1 = tfa.metrics.FBetaScore(num_classes=1)
  AUROC = tf.keras.metrics.AUC(curve='ROC', name = 'AUROC')
  AUPRC = tf.keras.metrics.AUC(curve='PR', name = 'AUPRC')
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy', F1, AUROC, AUPRC])
  keras.utils.plot_model(model,'model.png',show_layer_activations=True,show_shapes=True)
  # model.summary()
  return model

with tpu_strategy.scope():
  model = create_model()


In [ ]:
model.fit(train_dataset, validation_data = test_dataset, epochs=100)

Epoch 1/100
79/79 [==============================] - 80s 231ms/step - loss: 0.6716 - accuracy: 0.6750 - fbeta_score: 0.8185 - AUROC: 0.5248 - AUPRC: 0.7136 - val_loss: 2.0419 - val_accuracy: 0.5003 - val_fbeta_score: 0.6669 - val_AUROC: 0.5437 - val_AUPRC: 0.5328
Epoch 2/100
79/79 [==============================] - 9s 106ms/step - loss: 0.6090 - accuracy: 0.6923 - fbeta_score: 0.8185 - AUROC: 0.5897 - AUPRC: 0.7545 - val_loss: 0.7544 - val_accuracy: 0.5003 - val_fbeta_score: 0.6669 - val_AUROC: 0.5817 - val_AUPRC: 0.5765
Epoch 3/100
79/79 [==============================] - 8s 97ms/step - loss: 0.5798 - accuracy: 0.6942 - fbeta_score: 0.8185 - AUROC: 0.6695 - AUPRC: 0.8124 - val_loss: 0.7060 - val_accuracy: 0.5003 - val_fbeta_score: 0.6669 - val_AUROC: 0.5039 - val_AUPRC: 0.5356
Epoch 4/100
79/79 [==============================] - 9s 108ms/step - loss: 0.5502 - accuracy: 0.6933 - fbeta_score: 0.8185 - AUROC: 0.7215 - AUPRC: 0.8529 - val_loss: 0.6504 - val_accuracy: 0.5003 - val_fbeta_sc

## Nima's model

In [ ]:
def create_model():
    i = keras.Input((1024,12), name='signal')
    x = layers.Conv1D(filters=72, kernel_size=15, activation='swish', padding='same')(i)
    a = layers.Conv1D(filters=72, kernel_size=1, padding='same')(i)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    x = layers.SpatialDropout1D(0.2)(a)
    b = layers.AveragePooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=144, kernel_size=3, activation='swish', padding='same')(b)
    a = layers.Conv1D(filters=144, kernel_size=1, padding='same')(b)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    x = layers.SpatialDropout1D(0.2)(a)
    b = layers.AveragePooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=288, kernel_size=5, activation='swish', padding='same')(b)
    a = layers.Conv1D(filters=288, kernel_size=1, padding='same')(b)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    x = layers.SpatialDropout1D(0.2)(a)
    b = layers.AveragePooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=576, kernel_size=7, activation='swish', padding='same')(b)
    a = layers.Conv1D(filters=576, kernel_size=1, padding='same')(b)
    a = layers.Add()([a,x])
    a = layers.Activation('swish')(a)
    b = layers.SpatialDropout1D(0.2)(a)
    x = layers.GlobalAveragePooling1D()(b)

    # j = keras.Input((2000,2*2), name='fft')
    # y = layers.Conv1D(filters=72, kernel_size=3, activation='swish', padding='same')(j)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=144, kernel_size=5, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=288, kernel_size=7, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=576, kernel_size=9, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.AveragePooling1D(pool_size=2)(y)
    # y = layers.Conv1D(filters=1152, kernel_size=11, activation='swish', padding='same')(y)
    # y = layers.SpatialDropout1D(0.1)(y)
    # y = layers.GlobalAveragePooling1D()(y)

    # f = layers.Concatenate()([x,y])
    # f = layers.Activation('swish')(f)

    f = layers.Dense(576, activation='swish')(x)
    f = layers.Dropout(0.5)(f)
    f = layers.Dense(1,activation = 'sigmoid')(f)

    F1 = tfa.metrics.FBetaScore(num_classes=1)
    AUROC = tf.keras.metrics.AUC(curve='ROC', name = 'AUROC')
    AUPRC = tf.keras.metrics.AUC(curve='PR', name = 'AUPRC')
    model = keras.Model(i,f)
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy', AUROC, AUPRC])
    tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_dtype=False,show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,layer_range=None, show_layer_activations=False)
    return model

with tpu_strategy.scope():
  model = create_model()


In [ ]:
tf.keras.utils.plot_model(model,'/content/drive/MyDrive/FYP_ML/model_nima.png')

In [ ]:
weight_for_0 = (1 / (tot-pos)) * (tot / 2.0)
weight_for_1 = (1 / pos) * (tot / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(train_dataset, validation_data = test_dataset, epochs=100)

Epoch 1/100
313/313 [==============================] - 27s 53ms/step - loss: 0.6625 - accuracy: 0.6951 - AUROC: 0.6192 - AUPRC: 0.7604 - val_loss: 0.7571 - val_accuracy: 0.6066 - val_AUROC: 0.7229 - val_AUPRC: 0.6840
Epoch 2/100
313/313 [==============================] - 13s 39ms/step - loss: 0.5784 - accuracy: 0.7125 - AUROC: 0.6940 - AUPRC: 0.8190 - val_loss: 0.6676 - val_accuracy: 0.5860 - val_AUROC: 0.7716 - val_AUPRC: 0.7809
Epoch 3/100
313/313 [==============================] - 12s 38ms/step - loss: 0.5389 - accuracy: 0.7259 - AUROC: 0.7446 - AUPRC: 0.8631 - val_loss: 0.6169 - val_accuracy: 0.6231 - val_AUROC: 0.7892 - val_AUPRC: 0.7976
Epoch 4/100
313/313 [==============================] - 12s 37ms/step - loss: 0.5149 - accuracy: 0.7322 - AUROC: 0.7687 - AUPRC: 0.8830 - val_loss: 0.5492 - val_accuracy: 0.6617 - val_AUROC: 0.8139 - val_AUPRC: 0.8379
Epoch 5/100
313/313 [==============================] - 13s 38ms/step - loss: 0.4798 - accuracy: 0.7496 - AUROC: 0.8071 - AUPRC: 0.90

## Small model

In [ ]:

# import tensorflow_addons as tfa
# from sklearn.model_selection import train_test_split

model=keras.Sequential(name="BrainNeoCare_CNN")
model.add(keras.layers.Conv2D(filters=32, kernel_size=(1,3), activation='relu', input_shape=(12,1024),name="Input_conv2D"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_1"))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(1,3), activation='relu',name="Conv2D_2"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_2"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(1,3), activation='relu',name="Conv2D_3"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_3"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(1,3), activation='relu',name="Conv2D_4"))
model.add(keras.layers.AveragePooling2D((1,2),name="AveragePooling2D_1"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',name="2D_F_Con"))
model.add(keras.layers.AveragePooling2D((1,2),name="AveragePooling2D_2"))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu',name="2D_F_Con_2"))
model.add(keras.layers.MaxPooling2D((1,2),name="AveragePooling2D_3"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu') )
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, name="binary_crossentropy",)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.summary()

Model: "BrainNeoCare_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_conv2D (Conv2D)       (None, 12, 1022, 32)      128       
                                                                 
 Maxpooling2D_1 (MaxPooling2  (None, 12, 511, 32)      0         
 D)                                                              
                                                                 
 Conv2D_2 (Conv2D)           (None, 12, 509, 32)       3104      
                                                                 
 Maxpooling2D_2 (MaxPooling2  (None, 12, 254, 32)      0         
 D)                                                              
                                                                 
 Conv2D_3 (Conv2D)           (None, 12, 252, 64)       6208      
                                                                 
 Maxpooling2D_3 (MaxPooling2  (None, 12, 126, 64) 

In [ ]:
model.fit(train_dataset,epochs=10,verbose=1)

## Data manipulation and loading

In [3]:
x=np.load('/content/drive/MyDrive/FYP_ML/zenodo_data.npy', mmap_mode='r')
y=np.load('/content/drive/MyDrive/FYP_ML/zenodo_labels.npy', mmap_mode='r')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

FileNotFoundError: ignored

In [6]:
x_train=np.load('/content/drive/MyDrive/FYP_ML/balanced_x_train_v1.npy', mmap_mode='r')
x_test=np.load('/content/drive/MyDrive/FYP_ML/balanced_x_test_v1.npy', mmap_mode='r')
y_train=np.load('/content/drive/MyDrive/FYP_ML/balanced_y_train_v1.npy', mmap_mode='r')
y_test=np.load('/content/drive/MyDrive/FYP_ML/balanced_y_test_v1.npy', mmap_mode='r')



In [ ]:
x

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(signal, label):
  feature = {
      'signal': _bytes_feature(signal),
      'label':_int64_feature(label)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
CT = 16
SIZE = len(x_train)//CT
#change the name of folder to fyp_train_v1 for V1 data
#change the name of folder to fyp_train_v2 for V2 data
#change the name of folder to fyp_train_v3 for V3 data

for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j,CT))
    CT2 = min(SIZE,len(x_train)-j*SIZE)
    with tf.io.TFRecordWriter('%.2i-%i_train.tfrec'%(j,CT2)) as writer:
        for k in range(CT2):
            signal = x_train[SIZE*j+k].tobytes()
            label = int(y_train[SIZE*j+k])
            example = serialize_example(signal,label)
            writer.write(example)
            if k%100==0: print(k,', ',end='')
    !gsutil -m cp -r *train.tfrec gs://tinyml/fyp_train_v1/
    !rm *train.tfrec


Writing TFRecord 0 of 16...
0 , 100 , 200 , 300 , 400 , 500 , 600 , Copying file://00-625_train.tfrec [Content-Type=application/octet-stream]...
- [1/1 files][ 58.6 MiB/ 58.6 MiB] 100% Done                                    
Operation completed over 1 objects/58.6 MiB.                                     

Writing TFRecord 1 of 16...
0 , 100 , 200 , 300 , 400 , 500 , 600 , Copying file://01-625_train.tfrec [Content-Type=application/octet-stream]...
- [1/1 files][ 58.6 MiB/ 58.6 MiB] 100% Done                                    
Operation completed over 1 objects/58.6 MiB.                                     

Writing TFRecord 2 of 16...
0 , 100 , 200 , 300 , 400 , 500 , 600 , Copying file://02-625_train.tfrec [Content-Type=application/octet-stream]...
- [1/1 files][ 58.6 MiB/ 58.6 MiB] 100% Done                                    
Operation completed over 1 objects/58.6 MiB.                                     

Writing TFRecord 3 of 16...
0 , 100 , 200 , 300 , 400 , 500 , 600 , Copy

In [ ]:
CT = 4
SIZE = len(x_test)//CT

for j in range(CT):
    print(); print('Writing TFRecord %i of %i...'%(j,CT))
    CT2 = min(SIZE,len(x_test)-j*SIZE)
    with tf.io.TFRecordWriter('%.2i-%i_test.tfrec'%(j,CT2)) as writer:
        for k in range(CT2):
            signal = x_test[SIZE*j+k].tobytes()
            label = int(y_test[SIZE*j+k])
            example = serialize_example(signal,label)
            writer.write(example)
            if k%100==0: print(k,', ',end='')
    !gsutil -m cp -r *test.tfrec gs://tinyml/fyp_test_v1/
    !rm *test.tfrec


Writing TFRecord 0 of 4...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , Copying file://00-849_test.tfrec [Content-Type=application/octet-stream]...
/
Operation completed over 1 objects/79.6 MiB.                                     

Writing TFRecord 1 of 4...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , Copying file://01-849_test.tfrec [Content-Type=application/octet-stream]...
- [1/1 files][ 79.6 MiB/ 79.6 MiB] 100% Done                                    
Operation completed over 1 objects/79.6 MiB.                                     

Writing TFRecord 2 of 4...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , Copying file://02-849_test.tfrec [Content-Type=application/octet-stream]...
- [1/1 files][ 79.6 MiB/ 79.6 MiB] 100% Done                                    
Operation completed over 1 objects/79.6 MiB.                                     

Writing TFRecord 3 of 4...
0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , Copying file://03-849_test.tfrec [Content-